This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# Load ODIM_H5 Volume data from German Weather Service

In this example, we obtain and read the latest 2 hours of available volumetric radar data from German Weather Service available at [opendata.dwd.de](https://opendata.dwd.de). Finally we do some plotting.

This retrieves 24 timesteps of the 10 sweeps (moments DBZH and VRADH) of the DWD volume scan of a distinct radar. This amounts to 240 data files which are combined into one volumetric Cf/Radial2 like xarray powered structure.

Exports to single file Odim_H5 and Cf/Radial2 format are shown at the end of this tutorial.

<div class="alert alert-warning">

**Note** <br>

The used `open_odim` implementation is based on [netcdf4](https://unidata.github.io/netcdf4-python/), [h5py](https://www.h5py.org/), [h5netcdf](https://github.com/shoyer/h5netcdf) and [Xarray](http://xarray.pydata.org/en/stable/). It claims multiple data files and presents them in a simple structure. See also the notebook [wradlib_odim_multi_file_dataset](wradlib_odim_multi_file_dataset.ipynb) for further details.

</div>

In [ ]:
import wradlib as wrl
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as pl
import numpy as np
import xarray as xr
try:
    get_ipython().magic("matplotlib inline")
except:
    pl.ion()
from wradlib.io.xarray import open_odim, CfRadial   

In [ ]:
import urllib3
import os
import io
import glob
import shutil
import datetime
from itertools import chain

## Download radar volumes of latest 2 hours from server using `wetterdienst`


`wetterdienst` is a neat package for easy retrieval of data primarily from DWD. For further information have a look at their [documentation](https://wetterdienst.readthedocs.io/).

In [ ]:
from wetterdienst.dwd.radar import DWDRadarValues, DWDRadarParameter, DWDRadarDate, DWDRadarDataFormat, DWDRadarDataSubset
from wetterdienst.dwd.radar.sites import DWDRadarSite

In [ ]:
start_date = datetime.datetime.utcnow()

# Horizontal Doppler Velocity
request_velocity = DWDRadarValues(
    parameter=DWDRadarParameter.SWEEP_VOL_VELOCITY_H,
    start_date=start_date - datetime.timedelta(hours=2),
    end_date=start_date,
    site=DWDRadarSite.ESS,
    fmt=DWDRadarDataFormat.HDF5,
    subset=DWDRadarDataSubset.POLARIMETRIC,
)

# Horizontal Reflectivity
request_reflectivity = DWDRadarValues(
    parameter=DWDRadarParameter.SWEEP_VOL_REFLECTIVITY_H,
    start_date=start_date - datetime.timedelta(hours=2),
    end_date=start_date,
    site=DWDRadarSite.ESS,fmt=DWDRadarDataFormat.HDF5,
    subset=DWDRadarDataSubset.POLARIMETRIC,
)

# Submit requests.
results = chain(request_velocity.query(), request_reflectivity.query())

## Acquire data as memory buffer

In [ ]:
# Collect in-memory data
files = []
for item in results:
    files.append(item.data)

## Read the data into xarray powered structure

In [ ]:
# Decode data using wradlib.
vol = wrl.io.open_odim(files)

## Inspect structure
### Root Group

In [ ]:
vol.root

In [ ]:
vol.root.sweep_fixed_angle

### Sweep Groups

In [ ]:
list(vol)

In [ ]:
vol[0]

## plot sweeps
### DBZH

In [ ]:
fig = pl.figure(figsize=(20, 30))
import matplotlib.gridspec as gridspec
gs = gridspec.GridSpec(4, 3, wspace=0.4, hspace=0.4)
for i, ts in enumerate(vol):
    swp = ts[0]
    swp.data.DBZH.pipe(wrl.georef.georeference_dataset).wradlib.plot(ax=gs[i], fig=fig)
    ax = pl.gca()
    ax.set_title(vol.root.sweep_fixed_angle[i].values)

### VRADH

In [ ]:
fig = pl.figure(figsize=(20, 30))
import matplotlib.gridspec as gridspec
gs = gridspec.GridSpec(4, 3, wspace=0.4, hspace=0.4)
for i, ts in enumerate(vol):
    swp = ts[0]
    swp.data.VRADH.pipe(wrl.georef.georeference_dataset).wradlib.plot(ax=gs[i], fig=fig)
    ax = pl.gca()
    ax.set_title(vol.root.sweep_fixed_angle[i].values)

### Plot single sweep using cartopy

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

map_trans = ccrs.AzimuthalEquidistant(central_latitude=vol.root.latitude.values, 
                                      central_longitude=vol.root.longitude.values)

In [ ]:
vol[-1].data

In [ ]:
map_proj = ccrs.AzimuthalEquidistant(central_latitude=vol.root.latitude.values, 
                                      central_longitude=vol.root.longitude.values)
pm = vol[-1][0].data.pipe(wrl.georef.georeference_dataset).DBZH.wradlib.plot_ppi(proj=map_proj)
ax = pl.gca()
ax.gridlines(crs=map_proj)
print(ax)

In [ ]:
map_proj = ccrs.Mercator(central_longitude=vol.root.longitude.values)
fig = pl.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection=map_proj)
pm = vol[-1][0].data.pipe(wrl.georef.georeference_dataset).DBZH.wradlib.plot_ppi(ax=ax)
ax.gridlines(draw_labels=True)

In [ ]:
fig = pl.figure(figsize=(10, 8))
proj=ccrs.AzimuthalEquidistant(central_latitude=vol.root.latitude.values, 
                               central_longitude=vol.root.longitude.values)
ax = fig.add_subplot(111, projection=proj)
pm = vol[-1][0].data.DBZH.wradlib.plot_ppi(ax=ax)
ax.gridlines()

### Inspect radar moments

The dataarrays can be accessed by key or by attribute. Each dataarray has the datasets dimensions and coordinates of it's parent dataset. There are attributes connected which are defined by Cf/Radial and/or ODIM_H5 standard.

In [ ]:
vol[-1][0].data.DBZH

In [ ]:
vol[-1][0].data.sweep_mode

In [ ]:
vol.root

# Plot Quasi Vertical Profile

In [ ]:
vol

In [ ]:
ts = vol[-1]
ts

In [ ]:
fig = pl.figure(figsize=(10, 4))
ax = fig.add_subplot(111)
ts.data.DBZH.median('azimuth').plot(x='time', vmin=-10, vmax=30, ax=ax)
ax.set_title(f"{np.datetime_as_string(ts.data.time[0].values, unit='D')}")
ax.set_ylim(0, 20000)

## Export to OdimH5

This exports the radar volume at given timestep including all moments into one ODIM_H5 compliant data file.

In [ ]:
vol.to_odim('dwd_odim.h5', timestep=0)

## Export to Cf/Radial2

This exports the radar volume at given timestep including all moments into one Cf/Radial2 compliant data file.

In [ ]:
vol.to_cfradial2('dwd_cfradial2.nc', timestep=0)

## Import again and check equality

Small time differences are possible so drop times before comparison

In [ ]:
try:
    vol1 = open_odim('dwd_odim.h5')
    vol2 = CfRadial('dwd_cfradial2.nc', dim0='azimuth')
    xr.testing.assert_equal(vol1.root, vol2.root)
    xr.testing.assert_equal(vol1[1][0].data.drop('time').rename({'rtime': 'time'}), vol2['sweep_1'])
    print("Equality Check successful")
except:
    print("Equality Check not successful")